In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from sklearn import svm

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [4]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

In [5]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

## SVM With PCA and With Feature Engineering

In [6]:
def scale(df, df_test):
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array2 = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array2, index=df_test.index, columns=df_test.columns)
    return df_scaled, df_test_scaled

In [7]:
files = ['MeanWeeklyImputed', 'KNNWeeklyImputed', 'MovingAverageWeeklyImputed', 'RegressionWeeklyImputed', 'MIIWeeklyImputed']

for file in files:
    df = read_csv(file+'_PCA_training_FE.csv')
    print(file)
    X_test = read_csv(file + '_PCA_test_FE.csv')

    y = pd.read_csv('sp500_target_regimes.csv', parse_dates=True)
    y.index = y['date'].values
    y = y[['regime']]
    y = y['regime']=='BEAR'
    y = pd.DataFrame (y, columns = ['regime'])
       
    
    X_train = df
    y_train = y.loc['2000-01-01':'2015-01-01']
    y_test = y.loc['2015-01-02':]
    
    #print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    X_train, X_test = scale(X_train, X_test)
    param_test1 = {
        'gamma':np.logspace(-2, 10, 13),
        'C':np.logspace(-9, 3, 13),
    }
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    gsearch1 = GridSearchCV(estimator = svm.SVC(kernel='rbf', C=1, gamma=1, class_weight={1:1/3}), 
                            param_grid = param_test1, 
                            scoring='f1',
                            n_jobs=4,
                            iid=False, 
                            cv=tscv)

    gsearch1.fit(X_train, y_train)
    print(gsearch1.best_params_, gsearch1.best_score_)
      

    model = svm.SVC(kernel='rbf', gamma=gsearch1.best_params_['gamma'], C=gsearch1.best_params_['C'], class_weight={1:2})
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    print("_______________________________________")
    print("______________Training_________________")
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")
    print("_______________Testing_________________")
    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")


MeanWeeklyImputed
{'C': 1.0, 'gamma': 0.01} 0.35595402829028294
_______________________________________
______________Training_________________
     True False
True    372    0
False   0    410
_______________________________________
F1 score 0.0
---------------------------------------
_______________Testing_________________
     True False
True    49    167
False   16    40
_______________________________________
F1 score 0.6512455516014235
---------------------------------------
KNNWeeklyImputed
{'C': 10.0, 'gamma': 0.1} 0.3782349896480331
_______________________________________
______________Training_________________
     True False
True    372    0
False   0    410
_______________________________________
F1 score 0.0
---------------------------------------
_______________Testing_________________
     True False
True    65    207
False   0    0
_______________________________________
F1 score 0.6142433234421365
---------------------------------------
MovingAverageWeeklyImputed
{'C':